# Exercício 2 - Enunciado

Considere-se de novo o algoritmo estendido de Euclides apresentado no TP2  mas usando o tipo dos inteiros e um parâmetro $N>0$
```
INPUT  a, b : Int
assume  a > 0 and b > 0 and a < N and b < N
r, r', s, s', t, t' = a, b, 1, 0, 0, 1
while r' != 0
  q = r div r'
  r, r', s, s', t, t' = r', r − q × r', s', s − q × s', t', t − q × t' 
OUTPUT r, s, t
```

Este exercício é dirigido à prova de correção do algoritmo estendido de Euclides

1. Construa a asserção lógica que representa a pós-condição do algoritmo. Note que a definição da função  $\gcd$  é   $\gcd(a,b)\;\equiv\; \min \{\,r > 0\,|\,\exists\,s,t\,\centerdot\, r = a*s+b*t\,\}$ .
2. Usando a metodologia do comando havoc para o ciclo, escreva o programa na linguagem dos comandos anotados (LPA). Codifique a pós-condição do algoritmo com um comando assert .
3. Construa codificações do programa LPA através de transformadores de predicados: “weakest pre-condition” e “strongest post-condition”. 
4. Prove a correção  do programa LPA em ambas as codificações.

# Exercício 2 - Solução

```python
assert r = gcd(a,b) and r_prime = 0 and gcd(a,b) = a * s + b * t
```

In [ ]:
from pysmt.shortcuts import *
from pysmt.typing import *

def prove(f):
    with Solver(name="z3") as s:
        s.add_assertion(Not(f))
        if s.solve():
            print("Failed to prove.")
        else:
            print("Proved.")

In [ ]:
# Grammar

"""
S := CODE $
CODE := EXPRS | CYCLES | PRECOND | POSTCOND | IFS | SKIP | | 
CYCLE := while CONDS do CODE CODE
IFS := if CONDS then CODE IF CODE
IF := else CODE | 
PRECOND := assume CONDS CODE
POSTCOND := assert CONDS CODE
CONDS := id sinal id COND CODE
COND := and CONDS | or CONDS | 
EXPRS := ATRIBS EXPR CODE
ATRIBS := id ATRIB atrib
ATRIB := virg id ATRIB | 
EXPRE := EXP | EXP , EXPRE
EXP := TERM EXP_OPS
EXP_OPS := op_a TERM EXP_OPS | 
TERM := FACTOR TERM_OPS
TERM_OPS := op_m FACTOR TERM_OPS | 
FACTOR := id | int | lparen EXPR rparen
SKIP := skip CODE
"""

# TOKENS
"""
op_a = ["+","-"]
op_m = ["*", "div"]
id = r"[a-zA-z]+['_]*[a-zA-z]*"
atrib = ":="
int = r'\d+'
sinal = [">=", ">", "<", "<=", "=", "!="]
lparen, rparen = "(", ")"
while = "while"
if = "if"
then = "then"
do = "do"
else = "else"
skip = "skip"
assume = "assume"
assert = "assert"
"""

def tokenizer(str):
    op_a = [r"+",r"-"]
    op_m = [r"*", r"div"]
    id = r"[a-zA-z]+['_]*[a-zA-z]*"
    atrib = r":="
    int = r'\d+'
    sinal = [r">=", r">", r"<", r"<=", r"=", r"!="]
    lparen, rparen = r"(", r")"
    while = r"while"
    if = r"if"
    then = r"then"
    do = r"do"
    else = r"else"
    skip = r"skip"
    assume = r"assume"
    assert = r"assert"
    if str in op_a:
        return 0;
    if str in op_m:
        return 1;
    aa